In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [11]:
#load data
df = pd.read_csv("cities.csv")
df.rename(columns = {"Unnamed: 0":"city"}, inplace = True)
df

,city,country,Lat,Lng,temp (f),humidity %,cloudiness %,wind (mph)
0,willowmore,ZA,-33.29,23.49,65.95,55,0,9.75
1,marawi,SD,18.48,31.82,68.14,26,0,13.33
2,attert,BE,49.75,5.79,37.99,96,0,3.51
3,berlevag,NO,70.86,29.09,35.60,82,0,23.04
4,busselton,AU,-33.65,115.33,54.00,97,23,4.94
...,...,...,...,...,...,...,...,...
536,petropavlovsk-kamchatskiy,RU,53.05,158.65,28.40,74,1,4.47
537,tadine,NC,-21.55,167.88,77.79,73,100,7.11
538,west wendover,US,40.74,-114.07,39.20,41,1,8.05
539,kudahuvadhoo,MV,2.67,72.89,82.54,72,100,6.55


In [31]:
#configure gmaps
gmaps.configure(api_key = gkey)
#use lat and lng as locations and humidity as the weight
locations = df[["Lat", "Lng"]]
humidity = df["humidity %"]
#add heatmap to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
heat_layer.max_intensity = 100
heat_layer.point_radius = 4


In [54]:
ideal_df = df.loc[(df["humidity %"] < 25), :]
ideal_df = ideal_df.loc[(ideal_df["wind (mph)"] < 10), :]
ideal_df = ideal_df.loc[(ideal_df["temp (f)"] < 80) & (ideal_df["temp (f)"] > 60), :]
ideal_df = ideal_df.loc[(ideal_df["cloudiness %"] < 25), :]
ideal_df

,city,country,Lat,Lng,temp (f),humidity %,cloudiness %,wind (mph)
40,karasburg,NaN,-28.02,18.75,77.99,15,0,9.35
201,silver city,US,32.77,-108.28,60.80,21,1,8.05
455,djibo,BF,14.10,-1.63,77.36,18,0,8.59
495,tessalit,ML,20.20,1.01,69.84,23,0,9.06
496,arlit,NE,18.74,7.39,71.11,23,0,8.95
